In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

print(torch.__version__)

1.0.1.post2


In [2]:
class LeNet5(nn.Module):
    """
    Input - 1x32x32
    C1 - 6@28x28 (5x5 kernel)
    tanh
    S2 - 6@14x14 (2x2 kernel, stride 2) Subsampling
    C3 - 16@10x10 (5x5 kernel, complicated shit)
    tanh
    S4 - 16@5x5 (2x2 kernel, stride 2) Subsampling
    C5 - 120@1x1 (5x5 kernel)
    F6 - 84
    tanh
    F7 - 10 (Output)
    """
    def __init__(self):
        super(LeNet5, self).__init__()

        self.c1 = nn.Conv2d(1, 6, kernel_size=(5, 5))
        self.c2 = nn.Conv2d(6, 16, kernel_size=(5, 5))
        self.c3 = nn.Conv2d(16, 120, kernel_size=(4, 4))
        self.f4 = nn.Linear(120, 84)
        self.f5 = nn.Linear(84, 10)
        self.out = nn.LogSoftmax(dim=-1)

    def forward(self, img, debug=False):
        out = F.relu(self.c1(img))
        out = F.max_pool2d(out, kernel_size=(2, 2), stride=2)

        out = F.relu(self.c2(out))        
        out = F.max_pool2d(out, kernel_size=(2, 2), stride=2)

        out = F.relu(self.c3(out))
        out = out.view(img.size()[0], -1)

        out = F.relu(self.f4(out))
        out = self.f5(out)
        out = self.out(out)

        return out

    def view_size(self, img):
        out = F.relu(self.c1(img))
        print('x > c1 : {}'.format(out.size()))
        out = F.max_pool2d(out, kernel_size=(2, 2), stride=2)
        print('c1 > s1 : {}'.format(out.size()))

        out = F.relu(self.c2(out))
        print('s1 > c2 : {}'.format(out.size()))
        out = F.max_pool2d(out, kernel_size=(2, 2), stride=2)
        print('c2 > s2 : {}'.format(out.size()))

        out = F.relu(self.c3(out))
        out = out.view(img.size()[0], -1)
        print('s2 > c3 : {}'.format(out.size()))

        out = F.relu(self.f4(out))
        print('c3 > f4 : {}'.format(out.size()))

        out = self.f5(out)
        print('f4 > f5 : {}'.format(out.size()))

        out = self.out(out)
        print('f5 > out : {}'.format(out.size()))

        return out
    
model = LeNet5()
print(model)

LeNet5(
  (c1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (c2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (c3): Conv2d(16, 120, kernel_size=(4, 4), stride=(1, 1))
  (f4): Linear(in_features=120, out_features=84, bias=True)
  (f5): Linear(in_features=84, out_features=10, bias=True)
  (out): LogSoftmax()
)


In [3]:
from config import torch_data_dir
from torchvision import datasets, transforms

batch_size = 128

transform_func = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))]
)

# dataset
train_dataset = datasets.MNIST(
    torch_data_dir, train=True, download=True,
    transform=transform_func)

test_dataset = datasets.MNIST(
    torch_data_dir, train=False, download=True,
    transform=transform_func)

# data loader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
for X, y in train_loader:
    break

In [5]:
X.size()

torch.Size([128, 1, 28, 28])

In [6]:
model(X).size()

torch.Size([128, 10])

In [7]:
def train(model, device, train_loader, loss_func, optimizer, epoch):
    def status(new_line=False):
        nl = '\n' if new_line else ''
        print('\rTrain Epoch: {}, ({:.2f} %)\tLoss: {:.6f}'.format(
            epoch, 100. * (batch_idx + 1) / len(train_loader), loss.item()), end=nl)

    model.train()
    for batch_idx, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            status()
    status(True)

    return model

def test(model, device, test_loader, loss_func):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_func(y_pred, y, reduction='sum').item() # sum up batch loss
            pred = y_pred.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, accuracy

In [8]:
cuda = False

use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# model
model = LeNet5().to(device)

# loss func: negative log likelihood
loss_func = F.nll_loss

# optimizer
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# train
max_epoch = 100
print('train with {}'.format(device))
for epoch in range(1, max_epoch + 1):
    model = train(model, device, train_loader, loss_func, optimizer, epoch)
    if epoch % 5 == 0:
        test_loss, accuracy = test(model, device, test_loader, loss_func)
        print('test loss = {:.4}, accuracy = {:.4}'.format(test_loss, accuracy), end='\n\n')

train with cpu
Train Epoch: 1, (100.00 %)	Loss: 0.149087
Train Epoch: 2, (100.00 %)	Loss: 0.171757
Train Epoch: 3, (100.00 %)	Loss: 0.070017
Train Epoch: 4, (100.00 %)	Loss: 0.189119
Train Epoch: 5, (100.00 %)	Loss: 0.136921
test loss = 0.06714, accuracy = 97.87

Train Epoch: 6, (100.00 %)	Loss: 0.065333
Train Epoch: 7, (100.00 %)	Loss: 0.033118
Train Epoch: 8, (100.00 %)	Loss: 0.013952
Train Epoch: 9, (100.00 %)	Loss: 0.036747
Train Epoch: 10, (100.00 %)	Loss: 0.016085
test loss = 0.04662, accuracy = 98.51

Train Epoch: 11, (100.00 %)	Loss: 0.078598
Train Epoch: 12, (100.00 %)	Loss: 0.012086
Train Epoch: 13, (100.00 %)	Loss: 0.024042
Train Epoch: 14, (100.00 %)	Loss: 0.037821
Train Epoch: 15, (100.00 %)	Loss: 0.077070
test loss = 0.0492, accuracy = 98.44

Train Epoch: 16, (100.00 %)	Loss: 0.026971
Train Epoch: 17, (100.00 %)	Loss: 0.016850
Train Epoch: 18, (100.00 %)	Loss: 0.019636
Train Epoch: 19, (100.00 %)	Loss: 0.030258
Train Epoch: 20, (100.00 %)	Loss: 0.014263
test loss = 0.0367

In [9]:
torch.save(model.state_dict(), 'lenet5_params.pt')